<a href="https://colab.research.google.com/github/Ashwin81456/emotion-classification-using-eeg-signals/blob/main/randomsearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import the required libraries
import os
import time
import pickle
import pandas as pd
import numpy as np
from google.colab import drive

#for feature extraction
from scipy.signal import welch
from scipy.integrate import simps

#classifier libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB


from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

import xgboost as xgb

from sklearn import model_selection
from sklearn.metrics import classification_report, confusion_matrix
import itertools
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve

In [2]:
def read_data(filename):
    x = pickle._Unpickler(open(filename, 'rb'))
    x.encoding = 'latin1'
    p = x.load()
    return p

In [3]:
#creating the file names of the dataset to load it
files = []
for n in range (1, 33):
    s = 's'
    if n < 10:
        s += '0'
    s += str(n)
    s+=str(".dat")
    files.append(s)
print(files)

['s01.dat', 's02.dat', 's03.dat', 's04.dat', 's05.dat', 's06.dat', 's07.dat', 's08.dat', 's09.dat', 's10.dat', 's11.dat', 's12.dat', 's13.dat', 's14.dat', 's15.dat', 's16.dat', 's17.dat', 's18.dat', 's19.dat', 's20.dat', 's21.dat', 's22.dat', 's23.dat', 's24.dat', 's25.dat', 's26.dat', 's27.dat', 's28.dat', 's29.dat', 's30.dat', 's31.dat', 's32.dat']


In [4]:
# 32x40 = 1280 trials for 32 participants
labels = []
data = []
drive.mount('/content/drive')
for i in files:
  filename = "/content/drive/My Drive/MajorProject/Dataset/" + i
  trial = read_data(filename)
  labels.append(trial['labels'])
  data.append(trial['data'])

Mounted at /content/drive


In [5]:
# Lets see the shapes of the raw data
labels = np.array(labels)
data = np.array(data)
print("Labels: ", labels.shape) # participants x videos x labels
print("Data: ", data.shape) # participants x videos x channels x data

Labels:  (32, 40, 4)
Data:  (32, 40, 40, 8064)


In [6]:
# Re-shape arrays into desired shapes
labels = labels.flatten()
labels = labels.reshape(1280, 4)

data = data.flatten()
data = data.reshape(1280, 40, 8064)

In [7]:
# Double-check the new arrays
#Here trial = participants x vidoes = 32 x 40 = 1280

print("Labels: ", labels.shape) # trial x label
print("Data: ", data.shape) # trial x channel x data

Labels:  (1280, 4)
Data:  (1280, 40, 8064)


In [8]:
#creating the dataframes for the labels
labelsDf = pd.DataFrame(labels)
print("printing the labels dataframe\n")
print(labelsDf)
print("\n\nDescribing the labels dataframe")
labelsDf.describe()

printing the labels dataframe

         0     1     2     3
0     7.71  7.60  6.90  7.83
1     8.10  7.31  7.28  8.47
2     8.58  7.54  9.00  7.08
3     4.94  6.01  6.12  8.06
4     6.96  3.92  7.19  6.05
...    ...   ...   ...   ...
1275  3.91  6.96  5.82  3.12
1276  2.81  6.13  6.06  1.04
1277  3.05  7.01  5.10  1.10
1278  3.99  7.17  4.85  1.00
1279  7.15  4.03  9.00  1.88

[1280 rows x 4 columns]


Describing the labels dataframe


,0,1,2,3
count,1280.000000,1280.000000,1280.000000,1280.000000
mean,5.254313,5.156711,5.382750,5.518133
std,2.130816,2.020499,2.096321,2.282780
min,1.000000,1.000000,1.000000,1.000000
25%,3.867500,3.762500,3.932500,3.960000
50%,5.040000,5.230000,5.240000,6.050000
75%,7.050000,6.950000,7.040000,7.090000
max,9.000000,9.000000,9.000000,9.000000


In [20]:
#giving names to the label columns
df_labels= pd.DataFrame({'valence': labels[:,0], 'arousal': labels[:,1], 'dominance': labels[:,2], 'liking': labels[:,3]})
print(labelsDf.describe())
# Function to calculate RMS
def calculate_rms(column):
    return np.sqrt(np.mean(column**2))

# Calculate RMS, skewness, and kurtosis for each column
rms_values = labelsDf.apply(calculate_rms)
skewness_values = labelsDf.apply(skew)
kurtosis_values = labelsDf.apply(kurtosis)

# Print the results
print("RMS Values:")
print(rms_values)
print("\nSkewness Values:")
print(skewness_values)
print("\nKurtosis Values:")
print(kurtosis_values)


                 0            1            2            3
count  1280.000000  1280.000000  1280.000000  1280.000000
mean      5.254313     5.156711     5.382750     5.518133
std       2.130816     2.020499     2.096321     2.282780
min       1.000000     1.000000     1.000000     1.000000
25%       3.867500     3.762500     3.932500     3.960000
50%       5.040000     5.230000     5.240000     6.050000
75%       7.050000     6.950000     7.040000     7.090000
max       9.000000     9.000000     9.000000     9.000000
RMS Values:
0    5.669623
1    5.538131
2    5.776255
3    5.971332
dtype: float64

Skewness Values:
0   -0.196734
1   -0.287835
2   -0.130006
3   -0.558101
dtype: float64

Kurtosis Values:
0   -0.883901
1   -0.710391
2   -0.780571
3   -0.753041
dtype: float64


In [21]:
#Dropping the Dominance and Liking columns
df_labels=df_labels.drop('dominance',axis=1)
df_labels=df_labels.drop('liking',axis=1)
# print(df_labels.describe())
print(df_labels)

# df = df.drop('B', axis=1)

      valence  arousal
0        7.71     7.60
1        8.10     7.31
2        8.58     7.54
3        4.94     6.01
4        6.96     3.92
...       ...      ...
1275     3.91     6.96
1276     2.81     6.13
1277     3.05     7.01
1278     3.99     7.17
1279     7.15     4.03

[1280 rows x 2 columns]
RMS Values:
valence    5.669623
arousal    5.538131
dtype: float64

Skewness Values:
valence   -0.196734
arousal   -0.287835
dtype: float64

Kurtosis Values:
valence   -0.883901
arousal   -0.710391
dtype: float64


# Separte Valence and Arousal to HAHV, LAHV, HALV, LVLV

In [11]:
import numpy as np

# # Create a sample DataFrame with 'valence' and 'arousal' columns
# np.random.seed(0)
# valence = np.random.uniform(1, 9, 1280)
# arousal = np.random.uniform(1, 9, 1280)
# data = {'valence': valence, 'arousal': arousal}
# df_valence_arousal = pd.DataFrame(data)

# Calculate the median value of arousal and valence column
arousal_median = df_labels['arousal'].median()
print(arousal_median)
valence_median = df_labels['valence'].median()
print(valence_median)

# Create a new DataFrame with the desired columns
df_result = pd.DataFrame(index=range(1280), columns=['HAHV', 'LAHV', 'HALV', 'LALV'])
df_result[['HAHV', 'LAHV', 'HALV', 'LALV']] = 0

# Apply the conditions
df_result.loc[(df_labels['valence'] >= valence_median) & (df_labels['arousal'] >= arousal_median), 'HAHV'] = 1
df_result.loc[(df_labels['arousal'] < arousal_median) & (df_labels['valence'] >= valence_median), 'LAHV'] = 1
df_result.loc[(df_labels['arousal'] >= arousal_median) & (df_labels['valence'] < valence_median), 'HALV'] = 1
df_result.loc[(df_labels['valence'] < valence_median) & (df_labels['arousal'] < arousal_median), 'LALV'] = 1

# Show the first few rows of the result DataFrame
# df_result.tail()
print(df_result)


5.23
5.04
      HAHV  LAHV  HALV  LALV
0        1     0     0     0
1        1     0     0     0
2        1     0     0     0
3        0     0     1     0
4        0     1     0     0
...    ...   ...   ...   ...
1275     0     0     1     0
1276     0     0     1     0
1277     0     0     1     0
1278     0     0     1     0
1279     0     1     0     0

[1280 rows x 4 columns]


Verify the data in 4 classes

In [12]:
# Check the number of 1s in each individual column
count_HAHV = df_result['HAHV'].sum()
count_LAHV = df_result['LAHV'].sum()
count_HALV = df_result['HALV'].sum()
count_LALV = df_result['LALV'].sum()

print(f"Number of 1s in HAHV: {count_HAHV}")
print(f"Number of 1s in LAHV: {count_LAHV}")
print(f"Number of 1s in HALV: {count_HALV}")
print(f"Number of 1s in LALV: {count_LALV}")

print(f"Total = {count_HAHV+count_LAHV+count_HALV+count_LALV}") # the total must be 1280


Number of 1s in HAHV: 358
Number of 1s in LAHV: 322
Number of 1s in HALV: 282
Number of 1s in LALV: 318
Total = 1280


In [13]:
# Function to check if each trial has positive or negative valence
def positive_valence(trial):
    return 1 if labels[trial,0] >= np.median(labels[:,0]) else 0
# Function to check if each trial has high or low arousal
def high_arousal(trial):
    return 1 if labels[trial,1] >= np.median(labels[:,1]) else 0

In [22]:
from scipy.stats import skew, kurtosis
# Convert all ratings to boolean values
labels_encoded = []
for i in range (len(labels)):
    labels_encoded.append([positive_valence(i), high_arousal(i)])
labels_encoded = np.reshape(labels_encoded, (1280, 2))
df_labels = pd.DataFrame(data=labels_encoded, columns=["High Valence", "High Arousal"])
print(df_labels.describe())
print(df_labels)


       High Valence  High Arousal
count   1280.000000   1280.000000
mean       0.531250      0.500000
std        0.499218      0.500195
min        0.000000      0.000000
25%        0.000000      0.000000
50%        1.000000      0.500000
75%        1.000000      1.000000
max        1.000000      1.000000
      High Valence  High Arousal
0                1             1
1                1             1
2                1             1
3                0             1
4                1             0
...            ...           ...
1275             0             1
1276             0             1
1277             0             1
1278             0             1
1279             1             0

[1280 rows x 2 columns]


In [15]:
# Dataset with only Valence column
df_valence = df_labels['High Valence']
# Dataset with only Arousal column
df_arousal = df_labels['High Arousal']
print(df_valence)
print(df_valence.shape)

0       1
1       1
2       1
3       0
4       1
       ..
1275    0
1276    0
1277    0
1278    0
1279    1
Name: High Valence, Length: 1280, dtype: int64
(1280,)


# FEATURE EXTRACTION USING WELCH'S METHOD

In [16]:
eeg_channels = np.array(["Fp1", "AF3", "F3", "F7", "FC5", "FC1", "C3", "T7", "CP5", "CP1", "P3", "P7", "PO3", "O1", "Oz", "Pz", "Fp2", "AF4", "Fz", "F4", "F8", "FC6", "FC2", "Cz", "C4", "T8", "CP6", "CP2", "P4", "P8", "PO4", "O2"])
peripheral_channels = np.array(["hEOG", "vEOG", "zEMG", "tEMG", "GSR", "Respiration belt", "Plethysmograph", "Temperature"])

In [17]:
eeg_data = []
for i in range (len(data)):
  for j in range (len(eeg_channels)):
    eeg_data.append(data[i,j])
eeg_data = np.reshape(eeg_data, (len(data), len(eeg_channels), len(data[0,0])))
print(eeg_data.shape)

(1280, 32, 8064)


In [23]:
peripheral_data = []
for i in range (len(data)):
  for j in range (32,len(data[0])):
    peripheral_data.append(data[i,j])
peripheral_data = np.reshape(peripheral_data, (len(data), len(peripheral_channels), len(data[0,0])))
print(peripheral_data.shape)

(1280, 8, 8064)


In [24]:
def bandpower(data, sf, band, window_sec=None, relative=False):
    band = np.asarray(band)
    low, high = band

    # Define window length
    if window_sec is not None:
        nperseg = window_sec * sf
    else:
        nperseg = (2 / low) * sf

    # Compute the modified periodogram (Welch)
    freqs, psd = welch(data, sf, nperseg=nperseg)

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find closest indices of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using Simpson's rule.
    bp = simps(psd[idx_band], dx=freq_res)

    if relative:
        bp /= simps(psd, dx=freq_res)
    return bp

In [25]:
def get_band_power(trial, channel, band):
  bd = (0,0)

  if (band == "theta"): # drownsiness, emotional connection, intuition, creativity
    bd = (4,8)
  elif (band == "alpha"): # reflection, relaxation
    bd = (8,12)
  elif (band == "beta"): # concentration, problem solving, memory
    bd = (12,30)
  elif (band == "gamma"): # cognition, perception, learning, multi-tasking
    bd = (30,50)

  return bandpower(eeg_data[trial,channel], 128, bd)


In [26]:
print(get_band_power(0,31,"theta"))
print(get_band_power(0,31,"alpha"))
print(get_band_power(0,31,"beta"))
print(get_band_power(0,31,"gamma"))

5.434119660168186
5.369595513295193
6.286556266834863
1.1062125930881577


# Process new datasets with 6 EEG regions and 4 band power values

---



In [27]:
# Transform 1280x 32 x 8064 => 1280 x 128
eeg_band_arr = []
for i in range (len(eeg_data)):
  for j in range (len(eeg_data[0])):
    eeg_band_arr.append(get_band_power(i,j,"theta"))
    eeg_band_arr.append(get_band_power(i,j,"alpha"))
    eeg_band_arr.append(get_band_power(i,j,"beta"))
    eeg_band_arr.append(get_band_power(i,j,"gamma"))
eeg_band_arr = np.reshape(eeg_band_arr, (1280, 128))

In [28]:
frontal = np.array(["F3", "FC1", "Fz", "F4", "FC2"])
parietal = np.array(["P3", "P7", "Pz", "P4", "P8"])
occipital = np.array(["O1", "Oz", "O2", "PO3", "PO4"])
central = np.array(["CP5", "CP1", "Cz", "C4", "C3", "CP6", "CP2"])

In [29]:
eeg_theta = []
for i in range (len(eeg_data)):
  for j in range (len(eeg_data[0])):
    eeg_theta.append(get_band_power(i,j,"theta"))
eeg_theta = np.reshape(eeg_theta, (1280, 32))

df_theta = pd.DataFrame(data = eeg_theta, columns=eeg_channels)
print(df_theta.describe())

                Fp1           AF3            F3            F7           FC5  \
count   1280.000000   1280.000000   1280.000000   1280.000000   1280.000000   
mean     517.431002    974.493856    795.093910   1515.004071    746.435950   
std     1165.295163   3147.555132   3020.711666   4990.544508   2188.101734   
min        2.698928      1.995398      1.820656      3.283107      1.311200   
25%       23.306027     17.117475     18.392335     30.827191     14.023436   
50%       65.468048     82.102529     60.241953     91.266539     48.704968   
75%      331.636624    304.949248    175.394835    248.308696    218.358248   
max    15524.135098  38122.870846  39431.320394  49272.793208  20182.668545   

               FC1            C3           T7           CP5          CP1  ...  \
count  1280.000000   1280.000000  1280.000000   1280.000000  1280.000000  ...   
mean    345.936665    486.095113   354.004358    664.656754   276.667812  ...   
std     717.328768   1497.636647   641.432373

In [30]:
# Transform 880 x 32 x 8064 => 880 x 32
eeg_alpha = []
for i in range (len(eeg_data)):
  for j in range (len(eeg_data[0])):
    eeg_alpha.append(get_band_power(i,j,"alpha"))
eeg_alpha = np.reshape(eeg_alpha, (1280, 32))

df_alpha = pd.DataFrame(data = eeg_alpha, columns=eeg_channels)
print(df_alpha.describe())

               Fp1           AF3            F3            F7          FC5  \
count  1280.000000   1280.000000   1280.000000   1280.000000  1280.000000   
mean    173.544174    333.184883    285.545436    573.725974   249.793720   
std     378.371554   1058.758333   1036.330438   1943.692827   702.436934   
min       2.770151      1.793012      1.724442      2.793554     0.975527   
25%      14.082681     10.737435     10.012318     15.888935    10.074042   
50%      33.713172     34.992442     30.026795     38.178299    20.079100   
75%     125.508396    114.334883     73.488279     93.987498    74.040939   
max    5627.906982  12380.702125  12764.724842  20843.070851  6575.781434   

               FC1           C3           T7          CP5          CP1  ...  \
count  1280.000000  1280.000000  1280.000000  1280.000000  1280.000000  ...   
mean    138.011594   170.423962   124.101183   232.750895    97.610644  ...   
std     275.218934   497.507620   200.171342   735.717912   166.43002

In [31]:
# Transform 880 x 32 x 8064 => 880 x 32
eeg_beta = []
for i in range (len(eeg_data)):
  for j in range (len(eeg_data[0])):
    eeg_beta.append(get_band_power(i,j,"beta"))
eeg_beta = np.reshape(eeg_beta, (1280, 32))

df_beta = pd.DataFrame(data = eeg_beta, columns=eeg_channels)
print(df_beta.describe())

               Fp1          AF3           F3            F7          FC5  \
count  1280.000000  1280.000000  1280.000000   1280.000000  1280.000000   
mean     92.200266   200.684361   193.372441    341.951096   140.642394   
std     176.162562   668.027842   668.771074   1274.679748   386.740178   
min       4.486703     2.650970     2.784376      3.363422     2.788531   
25%      17.999409    13.190613    10.170223     17.951892    11.341571   
50%      34.413828    29.155242    28.120517     33.617573    22.402198   
75%      80.774739   107.619088    59.807546     62.087123    53.162211   
max    3528.485435  5784.160193  5841.475721  14848.700463  3094.687504   

               FC1           C3           T7          CP5          CP1  ...  \
count  1280.000000  1280.000000  1280.000000  1280.000000  1280.000000  ...   
mean     99.397187    96.901859    77.540922   137.387625    55.047411  ...   
std     197.556641   247.621839    90.107877   409.366974    85.846400  ...   
min     

In [32]:
# Transform 880 x 32 x 8064 => 880 x 32
eeg_gamma = []
for i in range (len(eeg_data)):
  for j in range (len(eeg_data[0])):
    eeg_gamma.append(get_band_power(i,j,"gamma"))
eeg_gamma = np.reshape(eeg_gamma, (1280, 32))

df_gamma = pd.DataFrame(data = eeg_gamma, columns=eeg_channels)
print(df_gamma.describe())

               Fp1          AF3           F3           F7          FC5  \
count  1280.000000  1280.000000  1280.000000  1280.000000  1280.000000   
mean     42.744494   124.741147   124.091979   108.925067    76.442367   
std     107.499857   527.124450   518.146548   429.569821   278.128003   
min       1.182047     1.207390     0.791623     1.263688     0.740460   
25%       7.596436     5.115142     4.041404     6.949604     4.479457   
50%      16.007283    12.358713    10.747773    13.883780    10.673231   
75%      35.665384    39.426894    24.629586    29.848906    32.491367   
max    2444.608696  4721.132009  4727.748382  5070.652851  2681.042953   

               FC1           C3           T7          CP5          CP1  ...  \
count  1280.000000  1280.000000  1280.000000  1280.000000  1280.000000  ...   
mean     50.838934    47.149484    36.843729    65.377132    24.880481  ...   
std     113.483233   143.272024    49.815122   221.091450    54.998736  ...   
min       0.76721

In [33]:
# Split the data into training/testing sets
def split_train_test(x, y):
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42)
  return x_train, x_test, y_train, y_test

In [34]:
# Feature scaling
def feature_scaling(train, test):
  sc = StandardScaler()
  train = sc.fit_transform(train)
  test = sc.transform(test)
  return train, test

In [35]:
band_names = np.array(["theta", "alpha", "beta", "gamma"])
channel_names = np.array(["frontal",  "central", "parietal", "occipital"])
label_names = np.array(["valence", "arousal"])

In [36]:
# Testing different kernels (linear, sigmoid, rbf, poly) to select the most optimal one
clf_svm = SVC(kernel = 'rbf',gamma='auto',random_state = 42, probability=True)

In [37]:
# Testing different k (odd) numbers, algorithm (auto, ball_tree, kd_tree) and weight (uniform, distance) to select the most optimal one
# clf_knn = KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='auto')
# Best parameters: {'algorithm': 'auto', 'leaf_size': 5, 'metric': 'chebyshev', 'n_neighbors': 9, 'p': 1, 'weights': 'uniform'}
# clf_knn = KNeighborsClassifier(algorithm='auto',leaf_size=5,metric='chebyshev',n_neighbors=9,p=1,weights='uniform')
from sklearn.model_selection import GridSearchCV

# without pipeline
knn = KNeighborsClassifier()
# Define parameter grid for GridSearchCV
param_grid = {
    'knn__n_neighbors': [3, 5, 7, 9, 11],  # Number of neighbors to consider
    'knn__weights': ['uniform', 'distance'],  # Weight function used in prediction
    'knn__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Algorithm used to compute the nearest neighbors
    'knn__leaf_size': [5,10,15,20,25,30, 40, 50],  # Leaf size passed to BallTree or KDTree
    'knn__p': [1, 2],  # Power parameter for Minkowski metric
    'knn__metric': ['euclidean', 'manhattan', 'chebyshev','minkowski']  # Distance metric
}

# Perform GridSearchCV
clf_knn = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')


In [38]:
clf_dtree=DecisionTreeClassifier(max_depth=100,criterion='entropy',min_samples_split=25,min_samples_leaf=15,splitter='best',max_features='sqrt')
# dt = DecisionTreeClassifier(random_state=42)

# # Define parameter grid
# param_grid = {
#     'criterion': ['gini', 'entropy'],
#     'splitter': ['best', 'random'],
#     'max_depth': [None, 10, 20, 30, 40, 50],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'max_features': ['auto', 'sqrt', 'log2', None],
#     'max_leaf_nodes': [None, 10, 20, 30, 40, 50]
# }

# # Perform GridSearchCV with 5-fold cross-validation
# clf_dtree = GridSearchCV(dt, param_grid, cv=5, scoring='accuracy')

In [40]:
# clf_rf=RandomForestClassifier(n_estimators=50,max_depth=20,min_samples_split=5)
# clf_rf=RandomForestClassifier(n_estimators=50,max_depth=20,criterion='entropy',min_samples_split=10,min_samples_leaf=5,max_features='sqrt')
                                        # Placeholder for PCA, n_components will be set via GridSearchCV
rf= RandomForestClassifier()

# Define parameter grid for GridSearchCV
param_grid = {
    'pca__n_components': [10, 20, 30, 40, 50,60,70,80,90,100],  # Adjust based on the dataset
    'rf__n_estimators': [50, 100, 150, 200],  # Number of trees in the forest
    'rf__max_features': ['auto', 'sqrt', 'log2'],  # Number of features to consider at each split
    'rf__max_depth': [None, 10, 15, 20, 25, 30],  # Maximum depth of the tree
    'rf__min_samples_split': [2, 5, 8, 10],  # Minimum number of samples required to split an internal node
    'rf__min_samples_leaf': [1, 2, 4,6,8],  # Minimum number of samples required to be at a leaf node
    'rf__bootstrap': [True, False]  # Whether bootstrap samples are used when building trees
}

# Perform GridSearchCV
clf_rf = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')


In [39]:
# clf_nb= GaussianNB()
clf_nb= GaussianNB(var_smoothing=1e-5)


In [41]:
# Testing different learning rate (alpha), solver (adam, sgd, lbfgs) and activation (relu, tanh, logistic) to select the most optimal one
# clf_mlp = MLPClassifier(solver='sgd', activation='relu', alpha=0.3, max_iter=600)
mlp = MLPClassifier()

# Define parameter grid
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'max_iter': [200, 400, 600, 800, 1000]

}

# Perform GridSearchCV with 5-fold cross-validation
clf_mlp = GridSearchCV(mlp, param_grid, cv=5, scoring='accuracy')

In [42]:
# clf_adaboost = AdaBoostClassifier(n_estimators=50, random_state=42)
from sklearn.tree import DecisionTreeClassifier

base_estimator=DecisionTreeClassifier()
ada = AdaBoostClassifier(estimator=base_estimator, random_state=42)

# Define parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200, 300],  # Number of boosting stages to be run
    'learning_rate': [0.001, 0.01, 0.1],  # Learning rate shrinks the contribution of each classifier
    'algorithm': ['SAMME', 'SAMME.R'],  # Algorithm used for boosting
    'estimator__criterion': ['gini', 'entropy'],  # Criterion for the base estimator
    'estimator__splitter': ['best', 'random'],  # Split strategy for the base estimator
    'estimator__max_depth': [5, 10,15,20,25],  # Maximum depth of the base estimator
    'estimator__min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
    'estimator__min_samples_leaf': [2,4,7],  # Minimum samples required to be at a leaf node
    'estimator__max_features': ['auto', 'sqrt', 'log2']  # Number of features to consider when looking for the best split
}

# Perform GridSearchCV
clf_adaboost = GridSearchCV(estimator=ada, param_grid=param_grid, cv=5, scoring='accuracy')


In [43]:
clf_xgb = xgb.XGBClassifier(learning_rate=0.1, max_depth=3, n_estimators=100, random_state=42)


In [46]:
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV


# clf_lgbm = lgb.LGBMClassifier()

classifier = lgb.LGBMClassifier()

# Define the parameter grid
param_grid = {
    'num_leaves': [31, 50, 70, 100],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 500, 1000],
    'max_depth': [3, 5, 8, -1],  # -1 means no limit
    'min_split_gain': [0, 0.01, 0.1],
    'min_child_weight': [1, 5, 10],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0, 0.1, 0.5]
}

# Initialize the RandomizedSearchCV object
clf_lgbm = RandomizedSearchCV(
    estimator=classifier,
    param_distributions=param_grid,
    n_iter=100,  # Number of parameter settings sampled
    scoring='accuracy',  # Evaluation metric
    cv=3,  # 3-fold cross-validation
    verbose=2,
    random_state=42,
    n_jobs=-1  # Use all available cores
)

In [47]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
clf_gpc = GaussianProcessClassifier(kernel=1.0 * RBF(length_scale=1.0), random_state=42)


In [48]:
from sklearn.linear_model import Perceptron
clf_perceptron = Perceptron(random_state=42)

In [49]:
!pip install catboost
import catboost as cb
clf_catboost = cb.CatBoostClassifier(iterations=1000, depth=6, learning_rate=0.1, loss_function='Logloss', verbose=0)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 1.8 MB/s eta 0:00:00


In [50]:
from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression(random_state=42)

In [51]:
models = []
models.append(('SVM', clf_svm))
models.append(('k-NN', clf_knn))
models.append(('DT', clf_dtree))
models.append(('RF', clf_rf))
models.append(('NB', clf_nb))
models.append(('MLP', clf_mlp))
models.append(('AB', clf_adaboost))
models.append(('XGB', clf_xgb))
models.append(('CB', clf_catboost))
models.append(('LGBM', clf_lgbm))
models.append(('GPC', clf_gpc))
models.append(('PER', clf_perceptron))
models.append(('LR',clf_lr))

In [57]:
def run_clf_cv(band, channel, label, clf):
  if (band == "theta"):
    df_x = df_theta
  elif (band == "alpha"):
    df_x = df_alpha
  elif (band == "beta"):
    df_x = df_beta
  elif (band == "gamma"):
    df_x = df_gamma

  if (channel == "frontal"):
    df_x = df_x[frontal]
  elif (channel == "central"):
    df_x = df_x[central]
  elif (channel == "parietal"):
    df_x = df_x[parietal]
  elif (channel == "occipital"):
    df_x = df_x[occipital]

  # df_y = df_arousal if (label == "arousal") else df_valence
  # print(df_y.shape)
  # df_y = df_result["HAHV"]
  # df_y = df_result
  # df_y = df_y.flatten
  # print(f"the shape of df_y is {df_y.shape}")
  # print(f"the shape of df_x is {df_x.shape}")
  if (label == "HAHV"):
    df_y = df_result["HAHV"]
  elif (label == "LAHV"):
    df_y = df_result["LAHV"]
  elif (label == "HALV"):
    df_y = df_result["HALV"]
  elif (label == "LALV"):
    df_y = df_result["LALV"]

  # print(f"the shape of df_y is {df_y.shape}")
  # print(f"the shape of df_x is {df_x.shape}")
  # Train-test split

  x_train, x_test, y_train, y_test = split_train_test(df_x, df_y)

  # Apply CV
  x_for_kfold = np.array(x_train)
  y_for_kfold = np.array(y_train)
  kfold = model_selection.KFold(n_splits=10)

  for i, j in kfold.split(x_for_kfold):
   x_train2, x_test2 = x_for_kfold[i], x_for_kfold[j]
   y_train2, y_test2 = y_for_kfold[i], y_for_kfold[j]

  # Feature scaling
  x_train2, x_test2 = feature_scaling(x_train2, x_test2)

  # Feature scaling
  # scaler = StandardScaler()
  # x_train2 = scaler.fit_transform(x_train2)
  # x_test2 = scaler.transform(x_test2)


  if (clf == "svm"):
    clf_svm.fit(x_train2, y_train2)
    y_predict = clf_svm.predict(x_test2)
  elif (clf == "knn"):
    clf_knn.fit(x_train2, y_train2)
    y_predict = clf_knn.predict(x_test2)
  elif (clf == "dtree"):
    clf_dtree.fit(x_train2, y_train2)
    y_predict = clf_dtree.predict(x_test2)
  elif (clf == "rf"):
    clf_rf.fit(x_train2, y_train2)
    y_predict = clf_rf.predict(x_test2)
  elif (clf == "nb"):
    clf_nb.fit(x_train2, y_train2)
    y_predict = clf_nb.predict(x_test2)
  elif (clf == "mlp"):
    clf_mlp.fit(x_train2, y_train2)
    y_predict = clf_mlp.predict(x_test2)
  elif (clf == "ab"):
    clf_adaboost.fit(x_train2, y_train2)
    y_predict = clf_adaboost.predict(x_test2)
  elif (clf == "xgb"):
    clf_xgb.fit(x_train2, y_train2)
    y_predict = clf_xgb.predict(x_test2)
  elif (clf == "lgbm"):
    clf_lgbm.fit(x_train2, y_train2)
    y_predict = clf_lgbm.predict(x_test2)
  elif (clf == "gpc"):
    clf_gpc.fit(x_train2, y_train2)
    y_predict = clf_gpc.predict(x_test2)
  elif (clf == "per"):
    clf_perceptron.fit(x_train2, y_train2)
    y_predict = clf_perceptron.predict(x_test2)
  elif (clf == "cb"):
    clf_catboost.fit(x_train2, y_train2)
    y_predict = clf_catboost.predict(x_test2)
  elif (clf == "lr"):
    clf_lr.fit(x_train2, y_train2)
    y_predict = clf_lr.predict(x_test2)

  return y_test2, y_predict

In [53]:
def get_accuracy(band, channel, label, clf):
  y_test2, y_predict = run_clf_cv(band, channel, label, clf)
  return np.round(accuracy_score(y_test2, y_predict)*100,2)

In [54]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
def print_conf(band, channel, label, clf):
  y_test2, y_predict = run_clf_cv(band, channel, label, clf)
  conf_matrix = confusion_matrix(y_test2, y_predict)
  # print(conf_matrix)
  plt.figure(figsize=(4, 2))  # Decrease the figure size
  plt.title('Confusion Matrix')
  sns.heatmap(conf_matrix,
              annot=True,
              fmt='g',
                 xticklabels=['Predicted Positive', 'Predicted Negative'],
                 yticklabels=['Actual Positive', 'Actual Negative'])

  # display matrix
  plt.ylabel('Actual',fontsize=12)
  plt.xlabel('Prediction',fontsize=12)
  plt.show()

  # printing the classification report aswell

  class_report = classification_report(y_test2, y_predict, target_names=['1', '0'])
  print("\nClassification Report:")
  print(class_report)

In [55]:
def print_accuracy(label, clf):
  arr = []
  for i in range (len(band_names)):
    for j in range (len(channel_names)):
      arr.append(get_accuracy(band_names[i], channel_names[j], label, clf))
  arr = np.reshape(arr, (4,4))
  df = pd.DataFrame(data = arr, index=band_names, columns=channel_names)

  #print("Top 3 EEG regions with highest scores")
  #print(df.apply(lambda s: s.abs()).max().nlargest(3))
  #print()
  #print("Top 2 bands with highest scores")
  #print(df.apply(lambda s: s.abs()).max(axis=1).nlargest(2))
  #print()
  #print("EEG region with highest scores per each band")
  #print(df.idxmax(axis=1))
  #print()
  #print("Accuracy Scores")
  #print(df.idxmax())
  #print()
  print(df)

In [ ]:
print_accuracy('HAHV', 'svm')

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [ ]:
print_accuracy('LAHV', 'svm')

In [ ]:
print_accuracy('HALV', 'svm')

In [ ]:
print_accuracy('LALV', 'svm')

In [ ]:
# print the confusion matrix which gives the highest accuracy
print_conf("theta","parietal","arousal","svm")

In [ ]:
print_accuracy('valence', 'svm')

In [ ]:
print_conf("gamma","central","valence","svm")

In [ ]:
print_accuracy('HAHV', 'ab')

In [ ]:
print_conf("gamma","occipital","arousal","ab")

In [ ]:
print_accuracy('valence', 'ab')

In [ ]:
print_accuracy('HAHV', 'xgb')

In [ ]:
print_accuracy('valence', 'xgb')

In [ ]:
print_conf("beta","central","valence","xgb")

In [ ]:
print_accuracy('HAHV', 'knn')

In [ ]:
print_accuracy('LALV', 'knn')

In [ ]:
print_accuracy('HALV', 'knn')

In [ ]:
print_accuracy('LAHV', 'knn')

In [ ]:
print_accuracy('HAHV', 'dtree')

In [ ]:
print_accuracy('LAHV', 'dtree')

In [ ]:
print_accuracy('HALV', 'dtree')

In [ ]:
print_accuracy('LALV', 'dtree')

In [ ]:
print_accuracy('HAHV', 'rf')

In [ ]:
print_accuracy('LAHV', 'rf')

In [ ]:
print_accuracy('HALV', 'rf')

In [ ]:
print_accuracy('LALV', 'rf')

In [ ]:
print_accuracy('valence', 'rf')

In [ ]:
print_accuracy('arousal', 'nb')

In [ ]:
print_accuracy('valence', 'nb')

In [ ]:
print_accuracy('HAHV', 'nb')

In [ ]:
print_accuracy('HALV', 'nb')

In [ ]:
print_accuracy('LAHV', 'nb')

In [ ]:
print_accuracy('LALV', 'nb')

In [ ]:
print_accuracy('HAHV', 'mlp')

In [ ]:
print_accuracy('HALV', 'mlp')

In [ ]:
print_accuracy('LAHV', 'mlp')

In [ ]:
print_accuracy('LALV', 'mlp')

In [ ]:
print_accuracy('valence', 'mlp')

In [ ]:
print_accuracy('arousal', 'lgbm')

In [ ]:
print_accuracy('valence', 'lgbm')

In [ ]:
print_accuracy('arousal', 'gpc')

In [ ]:
print_accuracy('valence', 'gpc')

In [ ]:
print_accuracy('arousal', 'per')

In [ ]:
print_accuracy('valence', 'per')

In [ ]:
print_accuracy('arousal', 'cb')

In [ ]:
print_conf("gamma","frontal","arousal","cb")

In [ ]:
print_accuracy('HAHV', 'cb')

In [ ]:
print_accuracy('HALV', 'cb')

In [ ]:
print_accuracy('LAHV', 'cb')

In [ ]:
print_accuracy('LALV', 'cb')

In [ ]:
print_accuracy('HAHV', 'lr')

In [ ]:
print_accuracy('HALV', 'lr')

In [ ]:
print_accuracy('LAHV', 'lr')

In [ ]:
print_accuracy('LALV', 'lr')

In [ ]:
print_accuracy('HAHV', 'ab')

In [ ]:
print_accuracy('HALV', 'ab')

In [ ]:
print_accuracy('LAHV', 'ab')

In [ ]:
print_accuracy('LALV', 'ab')

In [ ]:
print_accuracy('HAHV', 'per')

In [ ]:
print_accuracy('LAHV', 'per')

In [ ]:
print_accuracy('HALV', 'per')

In [ ]:
print_accuracy('LALV', 'per')

In [ ]:
print_conf('beta','parietal','HAHV','per')

In [ ]:
print_conf('alpha','parietal','LAHV','per')

In [ ]:
print_conf('beta','frontal','HALV','per')

In [ ]:
print_conf('alpha','central','LALV','per')

In [58]:
print_accuracy('HAHV', 'lgbm')

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[LightGBM] [Info] Number of positive: 266, number of negative: 714
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000108 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 980, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.271429 -> initscore=-0.987387
[LightGBM] [Info] Start training from score -0.987387
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [ ]:
print_accuracy('LAHV', 'lgbm')

In [59]:
print_accuracy('HALV', 'lgbm')

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[LightGBM] [Info] Number of positive: 222, number of negative: 758
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000117 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 980, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226531 -> initscore=-1.228006
[LightGBM] [Info] Start training from score -1.228006
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [ ]:
print_accuracy('LALV', 'lgbm')

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[LightGBM] [Info] Number of positive: 251, number of negative: 729
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000116 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 980, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.256122 -> initscore=-1.066221
[LightGBM] [Info] Start training from score -1.066221
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [ ]:
print_accuracy('HAHV', 'xgb')

In [ ]:
print_accuracy('LAHV', 'xgb')

In [ ]:
print_accuracy('HALV', 'xgb')

In [ ]:
print_accuracy('LALV', 'xgb')

In [ ]:
print_conf('gamma','central','HAHV','knn')

In [ ]:
print_conf('beta','frontal','LAHV','knn')

In [ ]:
print_conf('alpha','frontal','HALV','knn')

In [ ]:
print_conf('theta','parietal','LALV','knn')